# Calculate the damage sensitivity index

   ## Calculate the features of the state
       # ModeShapeCurvature Calculation
           # modes_prepared(modes_Modal_selected&modes_SSI_normalised) -->
             MSC_Modal&MSC_SSI2or MSC_SSI4
       # ModalStrainEnergy(fractional energy f) Calculation
           # modes_prepared(modes_Modal_selected&modes_SSI_normalised) -->
             MSE_f_Modal & MSE_f_SSI2 or MSE_f_SSI4
       # Flexibility Matrix(g)
           # modes_prepared(modes_Modal_selected&modes_SSI_normalised) -->
             g_Modal & g_SSI2 or g_SSI4
             
   ## Calculate the difference of the features of the state(damage Index)
       # Modes Shape difference Calculation
           # modes_prepared(modes_Modal_selected&modes_SSI_normalised) -->
           ##  modes_diff
           ##  modes_absabsdiff
           ##  modes_abssqudiff
       # ModeShapeCurvature difference Calculation
           # MSC_Modal&MSC_SSI2 or MSC_SSI4 -->
           ##  MSC_absiff_Modal&MSC_absdiff_SSI2 or MSC_absdiff_SSI4
           ##  MSC_absabsdiff_Modal&MSC_absabsdiff_SSI2 or MSC_absabsdiff_SSI4
           ##  MSC_abssqudiff_Modal&MSC_abssqudiff_SSI2 or MSC_abssqudiff_SSI4
             
       # ModalStrainEnergy fractional energy difference (division fikd/fik) Calculation
           # MSE_f_Modal & MSE_f_SSI2 MSE_f_SSI4 -->
             (fikd/fik)_Modal = MSE_f_Modal_damaged/MSE_f_Modal_undamaged
             (fikd/fik)_SSI2 = MSE_f_SSI2_damaged/MSE_f_SSI2_undamaged 
             (fikd/fik)_SSI4 = MSE_f_SSI4_damaged/MSE_f_SSI4_undamaged
                 # beta = sumofAllOrder((fikd/fik))
                     ## beta_Modal = sumofAllOrder((fikd/fik)_Modal)
                     ## beta_SSI2 = sumofAllOrder((fikd/fik)_SSI2)
                     ## beta_SSI4 = sumofAllOrder((fikd/fik)_SSI4)
                 # beta2 = sumofFirstTwoOrder((fikd/fik))
                     ## beta2_Modal = sumofFirstTwoOrder((fikd/fik)_Modal)
                     ## beta2_SSI2 = sumofFirstTwoOrder((fikd/fik)_SSI2)
                     ## beta2_SSI4 = sumofFirstTwoOrder((fikd/fik)_SSI4)
                 
       # Modal Flexibility Matrix 
           # g_Modal & g_SSI2or4 --> v_Modal & v_SSI2or4 --> vc_Modal & vc_SSI2or4 --> diffvc_Modal & diffvc_SSI2or4 
           ## absdiffvc_Modal & absdiffvc_SSI2or4 
           ## absabsdiffvc_Modal & absabsdiffvc_SSI2or4 
           ## abssqudiffvc_Modal & abssqudiffvc_SSI2or4 

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from scipy import interpolate

# Calculate the damage sensitivity index

   ## Calculate the features of the state
       # ModeShapeCurvature Calculation
           # modes_prepared(modes_Modal_selected&modes_SSI_normalised) -->
             MSC_Modal&MSC_SSI2or MSC_SSI4
       # ModalStrainEnergy(fractional energy f) Calculation
           # modes_prepared(modes_Modal_selected&modes_SSI_normalised) -->
             MSE_f_Modal & MSE_f_SSI2 or MSE_f_SSI4
       # Flexibility Matrix(g)
           # modes_prepared(modes_Modal_selected&modes_SSI_normalised) -->
             g_Modal & g_SSI2 or g_SSI4

In [2]:
dx = 25/200
L = 25
class Modes_prepared:
    def __init__(self,modes_preparedPath):
        self.modes_preparedPath = modes_preparedPath
        self.modes_p = np.loadtxt(self.modes_preparedPath,delimiter=',')
        self.order, self.sensornumber = self.modes_p.shape
        self.regionnumber = self.sensornumber-1
        # Geometry settings
        self.dx = 25/200
        self.L = 25
    def getModeShapeCurvature(self,ModeShapeCurvature_Path):
        self.MSC_Path = ModeShapeCurvature_Path
        if self.sensornumber == 20:
            self.deltax = 10*self.dx # this will change if we change the sensor number
        elif self.sensornumber == 10:
            self.deltax = 10*self.dx*2
        elif self.sensornumber == 6:
            self.deltax = 10*self.dx*3
        self.MSC = np.zeros_like(self.modes_p)
        for i in range(0,self.order):
            for j in range(1,self.sensornumber-1):
                self.MSC[i,j]=(self.modes_p[i,j+1]+self.modes_p[i,j-1]-2*self.modes_p[i,j])/np.square(self.deltax)
            # At the ends of the structure
            self.MSC[i,0]=(self.modes_p[i,2] + self.modes_p[i,0]-2*self.modes_p[i,1])/np.square(self.deltax)
            self.MSC[i,self.sensornumber-1] = (self.modes_p[i,self.sensornumber-1] + self.modes_p[i,self.sensornumber-3]-2*self.modes_p[i,self.sensornumber-2])/np.square(self.deltax)
        # save the modeShapeCurvature
        file = open(self.MSC_Path,'w')
        np.savetxt(file,self.MSC,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.MSC_Path))
        file.close()    
    def getFractionalEnergy(self,fractionalEnergyPath,xnew =np.linspace(5*dx,L,100) ):
        self.fractionalEnergyPath = fractionalEnergyPath
        if self.sensornumber == 20:
            self.Sensor_position=np.linspace(5*self.dx,self.L,20)
        elif self.sensornumber == 10:
            self.Sensor_position=np.linspace(5*self.dx,self.L,10)
        elif self.sensornumber == 6:
            self.Sensor_position=np.linspace(5*self.dx,self.L,6)
        self.xpts = self.Sensor_position
        self.ypts = self.modes_p
        self.xnew = xnew # new interpolation points
        self.intnumber = xnew.size
        self.times = self.intnumber//self.sensornumber
        self.tck = {}
        self.ynew = np.zeros((self.order,self.intnumber))
        self.yder2 = np.zeros((self.order,self.intnumber))
        self.yder2_squ = np.zeros((self.order,self.intnumber))
        self.f = np.zeros((self.order,self.regionnumber))
        for i in range(self.order):# loop over all modes order
            self.tck[i] = interpolate.splrep(self.xpts,self.ypts[i],s=0)
            self.ynew[i]=interpolate.splev(self.xnew,self.tck[i],der=0)
            self.yder2[i]=interpolate.splev(self.xnew,self.tck[i],der=2)
            self.yder2_squ[i] = np.square(self.yder2[i])
            for k in range(self.regionnumber): 
                self.f[i][k]=sum(self.yder2_squ[i][self.times*k:self.times*k+self.times-1])/sum(self.yder2_squ[i])
        self.f_sum = sum(self.f,0)# sum of fractional energy of  all order
        # save the fractional energy of each region and each order
        file = open(self.fractionalEnergyPath,'w')
        np.savetxt(file,self.f,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.fractionalEnergyPath))
        file.close()   
    def getFlexibilityMatrix(self,frequencyPath,gPath,GPath):
        self.freqs = np.loadtxt(frequencyPath,delimiter=',')
        self.freqs_squ = np.square(self.freqs)
        self.phi = np.array(self.modes_p)
        self.g = np.zeros(((self.order,self.sensornumber,self.sensornumber)))
        self.G = np.zeros((self.sensornumber,self.sensornumber))     
        for i in range(self.order):#loop over all modes order
            self.g[i] = np.outer(self.phi[i].T,self.phi[i])/self.freqs_squ[i]# each Order
            self.G = self.G+self.g[i]
        # save g 3d array
        np.save(gPath,self.g)
#         file = open(gPath,'w')
#         np.savetxt(file,self.g,fmt = '%s',delimiter = ',')
#         pipe = sys.stdout.write
#         pipe('  {} written Finished.\n'.format(gPath))
#         file.close() 
        # save G
        file = open(GPath,'w')
        np.savetxt(file,self.G,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(GPath))
        file.close()   

In [3]:
 # freqs_Modal are directly from the structural Health Monitoring tools
frequencyModalPathdic =  \
{0:'frequency_Modal\\undamaged_modal_frequencies.dat', \
1:'frequency_Modal\\damaged7_20%_modal_frequencies.dat', \
2:'frequency_Modal\\damaged7_50%_modal_frequencies.dat', \
3:'frequency_Modal\\damaged7_70%_modal_frequencies.dat'}

frequencySSIPathdic_Uppersensor_y =  \
{0:'frequency_SSI_order18\\undamaged_frequency_Uppersensor_y.dat', \
1:'frequency_SSI_order18\\damaged7_20%_frequency_Uppersensor_y.dat', \
2:'frequency_SSI_order18\\damaged7_50%_frequency_Uppersensor_y.dat', \
3:'frequency_SSI_order18\\damaged7_70%_frequency_Uppersensor_y.dat'}

frequencySSIPathdic_Uppersensor_y_less10 =  \
{0:'frequency_SSI_order18\\undamaged_frequency_Uppersensor_y_less10.dat', \
1:'frequency_SSI_order18\\damaged7_20%_frequency_Uppersensor_y_less10.dat', \
2:'frequency_SSI_order18\\damaged7_50%_frequency_Uppersensor_y_less10.dat', \
3:'frequency_SSI_order18\\damaged7_70%_frequency_Uppersensor_y_less10.dat'}


absdiff_freqs_ModalPathdic = { \
1:'DamageIndex\\freqs_Modal_Diff\\absdiff_damaged7_20%.dat', \
2:'DamageIndex\\freqs_Modal_Diff\\absdiff_damaged7_50%.dat', \
3:'DamageIndex\\freqs_Modal_Diff\\absdiff_damaged7_70%.dat' }

absdiff_freqs_SSI_order18Pathdic_Uppersensor_y = { \
1:'DamageIndex\\freqs_SSI_order18_Diff\\absdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\freqs_SSI_order18_Diff\\absdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\freqs_SSI_order18_Diff\\absdiff_damaged7_70%_Uppersensor_y.dat' }

absdiff_freqs_SSI_order18Pathdic_Uppersensor_y_less10 = { \
1:'DamageIndex\\freqs_SSI_order18_Diff\\absdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\freqs_SSI_order18_Diff\\absdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\freqs_SSI_order18_Diff\\absdiff_damaged7_70%_Uppersensor_y_less10.dat' }
# #freqsdiff calculation
# def AbsDiffCalculation(freqsDamagedPathdic,absdiffPathdic):
#     freqs_undamaged = np.loadtxt(freqsDamagedPathdic[0])
#     length = len(freqsDamagedPathdic)
#     freqs_damaged = {}
#     absdiff_freqs = {}
#     for i in range(1,length):
#         freqs_damaged[i] = np.loadtxt(freqsDamagedPathdic[i])
#         newsize = min(freqs_undamaged.size,freqs_damaged[i].size)
#         freqs_undamaged = freqs_undamaged[:newsize]
#         freqs_damaged[i] = freqs_damaged[i][:newsize]
#         absdiff_freqs[i] = abs(freqs_damaged[i] - freqs_undamaged)
#         file = open(absdiffPathdic[i],'w')
#         np.savetxt(file,absdiff_freqs[i],fmt = '%s',delimiter = ',')
#         pipe = sys.stdout.write
#         pipe('  {} written Finished.\n'.format(absdiffPathdic[i]))
#         file.close()  
# AbsDiffCalculation(frequencyModalPathdic,absdiff_freqs_ModalPathdic)
# AbsDiffCalculation(frequencySSIPathdic_Uppersensor_y,absdiff_freqs_SSI_order18Pathdic_Uppersensor_y)
# AbsDiffCalculation(frequencySSIPathdic_Uppersensor_y_less10,absdiff_freqs_SSI_order18Pathdic_Uppersensor_y_less10)

In [4]:
# Modes_prepared Path definition
## ModesModalSelected
modesModalSelectedPathdic_Uppersensor_y = { \
0:'modes_Modal_Selected\\undamaged_Uppersensor_y.dat', \
1:'modes_Modal_Selected\\damaged7_20%_Uppersensor_y.dat', \
2:'modes_Modal_Selected\\damaged7_50%_Uppersensor_y.dat', \
3:'modes_Modal_Selected\\damaged7_70%_Uppersensor_y.dat'}   
modesModalSelectedPathdic_Uppersensor_y_less10 = { \
0:'modes_Modal_Selected\\undamaged_Uppersensor_y_less10.dat', \
1:'modes_Modal_Selected\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'modes_Modal_Selected\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'modes_Modal_Selected\\damaged7_70%_Uppersensor_y_less10.dat'}  
## ModesSSI2
modesSSI2Pathdic_Uppersensor_y = \
{0:'modes_SSI2_order18\\undamaged_Uppersensor_y.dat', \
1:'modes_SSI2_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'modes_SSI2_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'modes_SSI2_order18\\damaged7_70%_Uppersensor_y.dat'}
modesSSI2Pathdic_Uppersensor_y_less10 = \
{0:'modes_SSI2_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'modes_SSI2_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'modes_SSI2_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'modes_SSI2_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
## ModesSSI4
modesSSI4Pathdic_Uppersensor_y = \
{0:'modes_SSI4_order18\\undamaged_Uppersensor_y.dat', \
1:'modes_SSI4_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'modes_SSI4_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'modes_SSI4_order18\\damaged7_70%_Uppersensor_y.dat'}
modesSSI4Pathdic_Uppersensor_y_less10 = \
{0:'modes_SSI4_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'modes_SSI4_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'modes_SSI4_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'modes_SSI4_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
modes_prepared_Modal_Uppersensor_y = {}
modes_prepared_Modal_Uppersensor_y_less10 = {}
modes_prepared_SSI2_Uppersensor_y = {}
modes_prepared_SSI2_Uppersensor_y_less10 = {}
modes_prepared_SSI4_Uppersensor_y = {}
modes_prepared_SSI4_Uppersensor_y_less10 = {}

In [5]:
# load modes_prepared_Uppersensor_y
for i in range(4):
    modes_prepared_Modal_Uppersensor_y[i] = Modes_prepared(modesModalSelectedPathdic_Uppersensor_y[i])
    modes_prepared_SSI2_Uppersensor_y[i] = Modes_prepared(modesSSI2Pathdic_Uppersensor_y[i])
    modes_prepared_SSI4_Uppersensor_y[i] = Modes_prepared(modesSSI4Pathdic_Uppersensor_y[i])
# load modes_prepared_Uppersensor_y_less10
for i in range(4):
    modes_prepared_Modal_Uppersensor_y_less10[i] = Modes_prepared(modesModalSelectedPathdic_Uppersensor_y_less10[i])
    modes_prepared_SSI2_Uppersensor_y_less10[i] = Modes_prepared(modesSSI2Pathdic_Uppersensor_y_less10[i])
    modes_prepared_SSI4_Uppersensor_y_less10[i] = Modes_prepared(modesSSI4Pathdic_Uppersensor_y_less10[i])

In [6]:
# ModeShapeCurvature_prepared Path definition
## ModeShapeCurvatureModalSelected
ModeShapeCurvatureModalPathdic_Uppersensor_y = { \
0:'ModeShapeCurvature_Modal\\undamaged_Uppersensor_y.dat', \
1:'ModeShapeCurvature_Modal\\damaged7_20%_Uppersensor_y.dat', \
2:'ModeShapeCurvature_Modal\\damaged7_50%_Uppersensor_y.dat', \
3:'ModeShapeCurvature_Modal\\damaged7_70%_Uppersensor_y.dat'}   
ModeShapeCurvatureModalPathdic_Uppersensor_y_less10 = { \
0:'ModeShapeCurvature_Modal\\undamaged_Uppersensor_y_less10.dat', \
1:'ModeShapeCurvature_Modal\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'ModeShapeCurvature_Modal\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'ModeShapeCurvature_Modal\\damaged7_70%_Uppersensor_y_less10.dat'}  
## ModeShapeCurvatureSSI2
ModeShapeCurvatureSSI2Pathdic_Uppersensor_y = \
{0:'ModeShapeCurvature_SSI2_order18\\undamaged_Uppersensor_y.dat', \
1:'ModeShapeCurvature_SSI2_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'ModeShapeCurvature_SSI2_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'ModeShapeCurvature_SSI2_order18\\damaged7_70%_Uppersensor_y.dat'}
ModeShapeCurvatureSSI2Pathdic_Uppersensor_y_less10 = \
{0:'ModeShapeCurvature_SSI2_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'ModeShapeCurvature_SSI2_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'ModeShapeCurvature_SSI2_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'ModeShapeCurvature_SSI2_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
## ModeShapeCurvatureSSI4
ModeShapeCurvatureSSI4Pathdic_Uppersensor_y = \
{0:'ModeShapeCurvature_SSI4_order18\\undamaged_Uppersensor_y.dat', \
1:'ModeShapeCurvature_SSI4_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'ModeShapeCurvature_SSI4_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'ModeShapeCurvature_SSI4_order18\\damaged7_70%_Uppersensor_y.dat'}
ModeShapeCurvatureSSI4Pathdic_Uppersensor_y_less10 = \
{0:'ModeShapeCurvature_SSI4_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'ModeShapeCurvature_SSI4_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'ModeShapeCurvature_SSI4_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'ModeShapeCurvature_SSI4_order18\\damaged7_70%_Uppersensor_y_less10.dat'}

In [7]:
# Calculate ModeShapeCurvature_Uppersensor_y
for i in range(4):
    modes_prepared_Modal_Uppersensor_y[i].getModeShapeCurvature(ModeShapeCurvatureModalPathdic_Uppersensor_y[i]) 
    modes_prepared_SSI2_Uppersensor_y[i].getModeShapeCurvature(ModeShapeCurvatureSSI2Pathdic_Uppersensor_y[i])
    modes_prepared_SSI4_Uppersensor_y[i].getModeShapeCurvature(ModeShapeCurvatureSSI4Pathdic_Uppersensor_y[i])
# Calculate ModeShapeCurvature_Uppersensor_y_less10
for i in range(4):
    modes_prepared_Modal_Uppersensor_y_less10[i].getModeShapeCurvature(ModeShapeCurvatureModalPathdic_Uppersensor_y_less10[i]) 
    modes_prepared_SSI2_Uppersensor_y_less10[i].getModeShapeCurvature(ModeShapeCurvatureSSI2Pathdic_Uppersensor_y_less10[i])
    modes_prepared_SSI4_Uppersensor_y_less10[i].getModeShapeCurvature(ModeShapeCurvatureSSI4Pathdic_Uppersensor_y_less10[i])

  ModeShapeCurvature_Modal\undamaged_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_SSI2_order18\undamaged_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_SSI4_order18\undamaged_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_Modal\damaged7_20%_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_SSI2_order18\damaged7_20%_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_SSI4_order18\damaged7_20%_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_Modal\damaged7_50%_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_SSI2_order18\damaged7_50%_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_SSI4_order18\damaged7_50%_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_Modal\damaged7_70%_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_SSI2_order18\damaged7_70%_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_SSI4_order18\damaged7_70%_Uppersensor_y.dat written Finished.
  ModeShapeCurvature_Modal\undamaged_Uppe

In [8]:
# Calculate the flexibility matrix
frequencyModalPathdic =  \
{0:'frequency_Modal\\undamaged_modal_frequencies.dat', \
1:'frequency_Modal\\damaged7_20%_modal_frequencies.dat', \
2:'frequency_Modal\\damaged7_50%_modal_frequencies.dat', \
3:'frequency_Modal\\damaged7_70%_modal_frequencies.dat'}

frequencySSI_order18Pathdic_Uppersensor_y =  \
{0:'frequency_SSI_order18\\undamaged_frequency_Uppersensor_y.dat', \
1:'frequency_SSI_order18\\damaged7_20%_frequency_Uppersensor_y.dat', \
2:'frequency_SSI_order18\\damaged7_50%_frequency_Uppersensor_y.dat', \
3:'frequency_SSI_order18\\damaged7_70%_frequency_Uppersensor_y.dat'}

frequencySSI_order18Pathdic_Uppersensor_y_less10 =  \
{0:'frequency_SSI_order18\\undamaged_frequency_Uppersensor_y_less10.dat', \
1:'frequency_SSI_order18\\damaged7_20%_frequency_Uppersensor_y_less10.dat', \
2:'frequency_SSI_order18\\damaged7_50%_frequency_Uppersensor_y_less10.dat', \
3:'frequency_SSI_order18\\damaged7_70%_frequency_Uppersensor_y_less10.dat'}


# Flexibility matrix Path definition
gModalPath_Uppersensor_y = { \
0:'FlexibilityMatrix_Modal\\g_undamaged_Uppersensor_y.npy', \
1:'FlexibilityMatrix_Modal\\g_damaged7_20%_Uppersensor_y.npy', \
2:'FlexibilityMatrix_Modal\\g_damaged7_50%_Uppersensor_y.npy', \
3:'FlexibilityMatrix_Modal\\g_damaged7_70%_Uppersensor_y.npy'}

GModalPath_Uppersensor_y = { \
0:'FlexibilityMatrix_Modal\\G_undamaged_Uppersensor_y.dat', \
1:'FlexibilityMatrix_Modal\\G_damaged7_20%_Uppersensor_y.dat', \
2:'FlexibilityMatrix_Modal\\G_damaged7_50%_Uppersensor_y.dat', \
3:'FlexibilityMatrix_Modal\\G_damaged7_70%_Uppersensor_y.dat'}

gSSI2Path_Uppersensor_y = { \
0:'FlexibilityMatrix_SSI2_order18\\g_undamaged_Uppersensor_y.npy', \
1:'FlexibilityMatrix_SSI2_order18\\g_damaged7_20%_Uppersensor_y.npy', \
2:'FlexibilityMatrix_SSI2_order18\\g_damaged7_50%_Uppersensor_y.npy', \
3:'FlexibilityMatrix_SSI2_order18\\g_damaged7_70%_Uppersensor_y.npy'}

GSSI2Path_Uppersensor_y = { \
0:'FlexibilityMatrix_SSI2_order18\\G_undamaged_Uppersensor_y.dat', \
1:'FlexibilityMatrix_SSI2_order18\\G_damaged7_20%_Uppersensor_y.dat', \
2:'FlexibilityMatrix_SSI2_order18\\G_damaged7_50%_Uppersensor_y.dat', \
3:'FlexibilityMatrix_SSI2_order18\\G_damaged7_70%_Uppersensor_y.dat'}

gSSI4Path_Uppersensor_y = { \
0:'FlexibilityMatrix_SSI4_order18\\g_undamaged_Uppersensor_y.npy', \
1:'FlexibilityMatrix_SSI4_order18\\g_damaged7_20%_Uppersensor_y.npy', \
2:'FlexibilityMatrix_SSI4_order18\\g_damaged7_50%_Uppersensor_y.npy', \
3:'FlexibilityMatrix_SSI4_order18\\g_damaged7_70%_Uppersensor_y.npy'}

GSSI4Path_Uppersensor_y = { \
0:'FlexibilityMatrix_SSI4_order18\\G_undamaged_Uppersensor_y.dat', \
1:'FlexibilityMatrix_SSI4_order18\\G_damaged7_20%_Uppersensor_y.dat', \
2:'FlexibilityMatrix_SSI4_order18\\G_damaged7_50%_Uppersensor_y.dat', \
3:'FlexibilityMatrix_SSI4_order18\\G_damaged7_70%_Uppersensor_y.dat'}

gModalPath_Uppersensor_y_less10 = { \
0:'FlexibilityMatrix_Modal\\g_undamaged_Uppersensor_y_less10.npy', \
1:'FlexibilityMatrix_Modal\\g_damaged7_20%_Uppersensor_y_less10.npy', \
2:'FlexibilityMatrix_Modal\\g_damaged7_50%_Uppersensor_y_less10.npy', \
3:'FlexibilityMatrix_Modal\\g_damaged7_70%_Uppersensor_y_less10.npy'}
GModalPath_Uppersensor_y_less10 = { \
0:'FlexibilityMatrix_Modal\\G_undamaged_Uppersensor_y_less10.dat', \
1:'FlexibilityMatrix_Modal\\G_damaged7_20%_Uppersensor_y_less10.dat', \
2:'FlexibilityMatrix_Modal\\G_damaged7_50%_Uppersensor_y_less10.dat', \
3:'FlexibilityMatrix_Modal\\G_damaged7_70%_Uppersensor_y_less10.dat'}

gSSI2Path_Uppersensor_y_less10 = { \
0:'FlexibilityMatrix_SSI2_order18\\g_undamaged_Uppersensor_y_less10.npy', \
1:'FlexibilityMatrix_SSI2_order18\\g_damaged7_20%_Uppersensor_y_less10.npy', \
2:'FlexibilityMatrix_SSI2_order18\\g_damaged7_50%_Uppersensor_y_less10.npy', \
3:'FlexibilityMatrix_SSI2_order18\\g_damaged7_70%_Uppersensor_y_less10.npy'}

GSSI2Path_Uppersensor_y_less10 = { \
0:'FlexibilityMatrix_SSI2_order18\\G_undamaged_Uppersensor_y_less10.dat', \
1:'FlexibilityMatrix_SSI2_order18\\G_damaged7_20%_Uppersensor_y_less10.dat', \
2:'FlexibilityMatrix_SSI2_order18\\G_damaged7_50%_Uppersensor_y_less10.dat', \
3:'FlexibilityMatrix_SSI2_order18\\G_damaged7_70%_Uppersensor_y_less10.dat'}


gSSI4Path_Uppersensor_y_less10 = { \
0:'FlexibilityMatrix_SSI4_order18\\g_undamaged_Uppersensor_y_less10.npy', \
1:'FlexibilityMatrix_SSI4_order18\\g_damaged7_20%_Uppersensor_y_less10.npy', \
2:'FlexibilityMatrix_SSI4_order18\\g_damaged7_50%_Uppersensor_y_less10.npy', \
3:'FlexibilityMatrix_SSI4_order18\\g_damaged7_70%_Uppersensor_y_less10.npy'}

GSSI4Path_Uppersensor_y_less10 = { \
0:'FlexibilityMatrix_SSI4_order18\\G_undamaged_Uppersensor_y_less10.dat', \
1:'FlexibilityMatrix_SSI4_order18\\G_damaged7_20%_Uppersensor_y_less10.dat', \
2:'FlexibilityMatrix_SSI4_order18\\G_damaged7_50%_Uppersensor_y_less10.dat', \
3:'FlexibilityMatrix_SSI4_order18\\G_damaged7_70%_Uppersensor_y_less10.dat'}

In [9]:
# Calculate ModeShapeCurvature_Uppersensor_y
#getFlexibilityMatrix(self,frequencyPath,gPath,GPath)
for i in range(4):
    modes_prepared_Modal_Uppersensor_y[i].getFlexibilityMatrix(frequencyModalPathdic[i],gModalPath_Uppersensor_y[i],GModalPath_Uppersensor_y[i]) 
    modes_prepared_SSI2_Uppersensor_y[i].getFlexibilityMatrix(frequencySSI_order18Pathdic_Uppersensor_y[i],gSSI2Path_Uppersensor_y[i],GSSI2Path_Uppersensor_y[i])
    modes_prepared_SSI4_Uppersensor_y[i].getFlexibilityMatrix(frequencySSI_order18Pathdic_Uppersensor_y[i],gSSI4Path_Uppersensor_y[i],GSSI4Path_Uppersensor_y[i])
# Calculate ModeShapeCurvature_Uppersensor_y_less10
for i in range(4):
    modes_prepared_Modal_Uppersensor_y_less10[i].getFlexibilityMatrix(frequencyModalPathdic[i],gModalPath_Uppersensor_y_less10[i],GModalPath_Uppersensor_y_less10[i]) 
    modes_prepared_SSI2_Uppersensor_y_less10[i].getFlexibilityMatrix(frequencySSI_order18Pathdic_Uppersensor_y_less10[i],gSSI2Path_Uppersensor_y_less10[i],GSSI2Path_Uppersensor_y_less10[i])
    modes_prepared_SSI4_Uppersensor_y_less10[i].getFlexibilityMatrix(frequencySSI_order18Pathdic_Uppersensor_y_less10[i],gSSI4Path_Uppersensor_y_less10[i],GSSI4Path_Uppersensor_y_less10[i])

  FlexibilityMatrix_Modal\G_undamaged_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_SSI2_order18\G_undamaged_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_SSI4_order18\G_undamaged_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_Modal\G_damaged7_20%_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_SSI2_order18\G_damaged7_20%_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_SSI4_order18\G_damaged7_20%_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_Modal\G_damaged7_50%_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_SSI2_order18\G_damaged7_50%_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_SSI4_order18\G_damaged7_50%_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_Modal\G_damaged7_70%_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_SSI2_order18\G_damaged7_70%_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_SSI4_order18\G_damaged7_70%_Uppersensor_y.dat written Finished.
  FlexibilityMatrix_Modal\G_u

In [10]:
# FractionalEnergy Path definition
## FractionalEnergyModalSelected
FractionalEnergyModalPathdic_Uppersensor_y = { \
0:'FractionalEnergy_Modal\\undamaged_Uppersensor_y.dat', \
1:'FractionalEnergy_Modal\\damaged7_20%_Uppersensor_y.dat', \
2:'FractionalEnergy_Modal\\damaged7_50%_Uppersensor_y.dat', \
3:'FractionalEnergy_Modal\\damaged7_70%_Uppersensor_y.dat'}   
FractionalEnergyModalPathdic_Uppersensor_y_less10 = { \
0:'FractionalEnergy_Modal\\undamaged_Uppersensor_y_less10.dat', \
1:'FractionalEnergy_Modal\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'FractionalEnergy_Modal\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'FractionalEnergy_Modal\\damaged7_70%_Uppersensor_y_less10.dat'}  
## FractionalEnergySSI2
FractionalEnergySSI2Pathdic_Uppersensor_y = \
{0:'FractionalEnergy_SSI2_order18\\undamaged_Uppersensor_y.dat', \
1:'FractionalEnergy_SSI2_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'FractionalEnergy_SSI2_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'FractionalEnergy_SSI2_order18\\damaged7_70%_Uppersensor_y.dat'}
FractionalEnergySSI2Pathdic_Uppersensor_y_less10 = \
{0:'FractionalEnergy_SSI2_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'FractionalEnergy_SSI2_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'FractionalEnergy_SSI2_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'FractionalEnergy_SSI2_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
## FractionalEnergySSI4
FractionalEnergySSI4Pathdic_Uppersensor_y = \
{0:'FractionalEnergy_SSI4_order18\\undamaged_Uppersensor_y.dat', \
1:'FractionalEnergy_SSI4_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'FractionalEnergy_SSI4_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'FractionalEnergy_SSI4_order18\\damaged7_70%_Uppersensor_y.dat'}
FractionalEnergySSI4Pathdic_Uppersensor_y_less10 = \
{0:'FractionalEnergy_SSI4_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'FractionalEnergy_SSI4_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'FractionalEnergy_SSI4_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'FractionalEnergy_SSI4_order18\\damaged7_70%_Uppersensor_y_less10.dat'}

In [11]:
# Calculate FractionalEnergy_Uppersensor_y
for i in range(4):
    modes_prepared_Modal_Uppersensor_y[i].getFractionalEnergy(FractionalEnergyModalPathdic_Uppersensor_y[i]) 
    modes_prepared_SSI2_Uppersensor_y[i].getFractionalEnergy(FractionalEnergySSI2Pathdic_Uppersensor_y[i])
    modes_prepared_SSI4_Uppersensor_y[i].getFractionalEnergy(FractionalEnergySSI4Pathdic_Uppersensor_y[i])
# Calculate FractionalEnergy_Uppersensor_y_less10
for i in range(4):
    modes_prepared_Modal_Uppersensor_y_less10[i].getFractionalEnergy(FractionalEnergyModalPathdic_Uppersensor_y_less10[i]) 
    modes_prepared_SSI2_Uppersensor_y_less10[i].getFractionalEnergy(FractionalEnergySSI2Pathdic_Uppersensor_y_less10[i])
    modes_prepared_SSI4_Uppersensor_y_less10[i].getFractionalEnergy(FractionalEnergySSI4Pathdic_Uppersensor_y_less10[i])

  FractionalEnergy_Modal\undamaged_Uppersensor_y.dat written Finished.
  FractionalEnergy_SSI2_order18\undamaged_Uppersensor_y.dat written Finished.
  FractionalEnergy_SSI4_order18\undamaged_Uppersensor_y.dat written Finished.
  FractionalEnergy_Modal\damaged7_20%_Uppersensor_y.dat written Finished.
  FractionalEnergy_SSI2_order18\damaged7_20%_Uppersensor_y.dat written Finished.
  FractionalEnergy_SSI4_order18\damaged7_20%_Uppersensor_y.dat written Finished.
  FractionalEnergy_Modal\damaged7_50%_Uppersensor_y.dat written Finished.
  FractionalEnergy_SSI2_order18\damaged7_50%_Uppersensor_y.dat written Finished.
  FractionalEnergy_SSI4_order18\damaged7_50%_Uppersensor_y.dat written Finished.
  FractionalEnergy_Modal\damaged7_70%_Uppersensor_y.dat written Finished.
  FractionalEnergy_SSI2_order18\damaged7_70%_Uppersensor_y.dat written Finished.
  FractionalEnergy_SSI4_order18\damaged7_70%_Uppersensor_y.dat written Finished.
  FractionalEnergy_Modal\undamaged_Uppersensor_y_less10.dat writt

# FirstPart calculate the damage sensitivity index
             
   ## Calculate the difference of the features of the state(damage Index)
       # Modes Shape difference Calculation
           # modes_prepared(modes_Modal_selected&modes_SSI_normalised) -->
             modes_diff
             modes_absabsdiff
             modes_abssqudiff
       # ModeShapeCurvature difference Calculation
           # MSC_Modal&MSC_SSI2or MSC_SSI4 -->
             MSC_absiff_Modal&MSC_absdiff_SSI2or MSC_absdiff_SSI4
             MSC_absabsdiff_Modal&MSC_absabsdiff_SSI2or MSC_absabsdiff_SSI4
             MSC_abssqudiff_Modal&MSC_abssqudiff_SSI2or MSC_abssqudiff_SSI4

In [12]:
class DamageSensitiveFeature:
    def __init__(self,DSF_undamaged_Path,DSF_damaged_Path):
        self.DSF_undamaged_Path = DSF_undamaged_Path
        self.DSF_damaged_Path = DSF_damaged_Path
        #load data undamaged case
        self.DamageSensitiveFeature_undamaged=np.loadtxt(self.DSF_undamaged_Path,delimiter = ',')
        #load data damaged case
        self.DamageSensitiveFeature_damaged=np.loadtxt(self.DSF_damaged_Path,delimiter = ',')
    def DiffIndexCalculation(self,absdiffPath,absabsdiffPath,abssqudiffPath):
        self.orderDamaged,self.sensornumber_damaged = self.DamageSensitiveFeature_damaged.shape
        self.orderUndamaged,self.sensornumber_undamaged = self.DamageSensitiveFeature_undamaged.shape       
        self.newOrder = min(self.orderDamaged,self.orderUndamaged)
        self.DamageSensitiveFeature_undamaged = self.DamageSensitiveFeature_undamaged[:self.newOrder,]
        self.DamageSensitiveFeature_damaged = self.DamageSensitiveFeature_damaged[:self.newOrder,]

        #damage index absdiff
        self.absdiff = abs(self.DamageSensitiveFeature_damaged-self.DamageSensitiveFeature_undamaged)
        self.sumOfabsdiff = sum(self.absdiff,0)
        # save the absdiff
        self.absdiffPath = absdiffPath
        f = open(self.absdiffPath,'w')
        np.savetxt(f,self.absdiff,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.absdiffPath))
        f.close()
        
        #damage index absabsdiff
        self.absabsdiff = abs(abs(self.DamageSensitiveFeature_damaged)-abs(self.DamageSensitiveFeature_undamaged))
        self.sumOfabsabsdiff = sum(self.absabsdiff,0)   
        # save the absabsdiff
        self.absabsdiffPath = absabsdiffPath
        f = open(self.absabsdiffPath,'w')
        np.savetxt(f,self.absabsdiff,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.absabsdiffPath))
        f.close()
        
        #damage index abssqudiff
        self.abssqudiff = abs(np.square(self.DamageSensitiveFeature_damaged)-np.square(self.DamageSensitiveFeature_undamaged))
        self.sumOfabssqudiff = sum(self.abssqudiff,0)
        # save the abssqudiff
        self.abssqudiffPath = abssqudiffPath
        f = open(self.abssqudiffPath,'w')
        np.savetxt(f,self.abssqudiff,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.abssqudiffPath))
        f.close()
    def DivisionIndexCalculation(self,betaPath,beta2Path):
        # sum of all order
        self.beta = sum(self.DamageSensitiveFeature_damaged,0)/sum(self.DamageSensitiveFeature_undamaged,0)
        # sum of first two order
        self.beta2 = sum(self.DamageSensitiveFeature_damaged[:2],0)/sum(self.DamageSensitiveFeature_undamaged[:2],0)
        # save beta
        self.betaPath = betaPath
        f = open(self.betaPath,'w')
        np.savetxt(f,self.beta,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.betaPath))
        f.close()
        # save beta2
        self.beta2Path = beta2Path
        f = open(self.beta2Path,'w')
        np.savetxt(f,self.beta2,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(self.beta2Path))
        f.close()
 

In [13]:
FractionalEnergyModalPathdic_Uppersensor_y = { \
0:'FractionalEnergy_Modal\\undamaged_Uppersensor_y.dat', \
1:'FractionalEnergy_Modal\\damaged7_20%_Uppersensor_y.dat', \
2:'FractionalEnergy_Modal\\damaged7_50%_Uppersensor_y.dat', \
3:'FractionalEnergy_Modal\\damaged7_70%_Uppersensor_y.dat'}
f_undamaged=np.loadtxt(FractionalEnergyModalPathdic_Uppersensor_y[0],delimiter = ',')
f_damaged=np.loadtxt(FractionalEnergyModalPathdic_Uppersensor_y[3],delimiter = ',')

In [14]:
# Modes_prepared Path definition
## ModesModalSelected
modesModalSelectedPathdic_Uppersensor_y = { \
0:'modes_Modal_Selected\\undamaged_Uppersensor_y.dat', \
1:'modes_Modal_Selected\\damaged7_20%_Uppersensor_y.dat', \
2:'modes_Modal_Selected\\damaged7_50%_Uppersensor_y.dat', \
3:'modes_Modal_Selected\\damaged7_70%_Uppersensor_y.dat'}   
modesModalSelectedPathdic_Uppersensor_y_less10 = { \
0:'modes_Modal_Selected\\undamaged_Uppersensor_y_less10.dat', \
1:'modes_Modal_Selected\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'modes_Modal_Selected\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'modes_Modal_Selected\\damaged7_70%_Uppersensor_y_less10.dat'}  
## ModesSSI2
modesSSI2_order18Pathdic_Uppersensor_y = \
{0:'modes_SSI2_order18\\undamaged_Uppersensor_y.dat', \
1:'modes_SSI2_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'modes_SSI2_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'modes_SSI2_order18\\damaged7_70%_Uppersensor_y.dat'}
modesSSI2_order18Pathdic_Uppersensor_y_less10 = \
{0:'modes_SSI2_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'modes_SSI2_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'modes_SSI2_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'modes_SSI2_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
## ModesSSI4
modesSSI4_order18Pathdic_Uppersensor_y = \
{0:'modes_SSI4_order18\\undamaged_Uppersensor_y.dat', \
1:'modes_SSI4_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'modes_SSI4_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'modes_SSI4_order18\\damaged7_70%_Uppersensor_y.dat'}
modesSSI4_order18Pathdic_Uppersensor_y_less10 = \
{0:'modes_SSI4_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'modes_SSI4_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'modes_SSI4_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'modes_SSI4_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
modes_prepared_Modal_Uppersensor_y = {}
modes_prepared_Modal_Uppersensor_y_less10 = {}
modes_prepared_SSI2_order18_Uppersensor_y = {}
modes_prepared_SSI2_order18_Uppersensor_y_less10 = {}
modes_prepared_SSI4_order18_Uppersensor_y = {}
modes_prepared_SSI4_order18_Uppersensor_y_less10 = {}

In [15]:
#Damage Index Path Definition
## modes_Modal_Diff_Uppersensor_y
absdiff_modes_Modal_Diff_Uppersensor_y = { \
1:'DamageIndex\\modes_Modal_Diff\\absdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\modes_Modal_Diff\\absdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\modes_Modal_Diff\\absdiff_damaged7_70%_Uppersensor_y.dat'}
absabsdiff_modes_Modal_Diff_Uppersensor_y = { \
1:'DamageIndex\\modes_Modal_Diff\\absabsdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\modes_Modal_Diff\\absabsdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\modes_Modal_Diff\\absabsdiff_damaged7_70%_Uppersensor_y.dat'}
abssqudiff_modes_Modal_Diff_Uppersensor_y = {  \
1:'DamageIndex\\modes_Modal_Diff\\abssqudiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\modes_Modal_Diff\\abssqudiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\modes_Modal_Diff\\abssqudiff_damaged7_70%_Uppersensor_y.dat'}
## modes_SSI2_Diff_order18_Uppersensor_y
absdiff_modes_SSI2_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\modes_SSI2_Diff_order18\\absdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\modes_SSI2_Diff_order18\\absdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\modes_SSI2_Diff_order18\\absdiff_damaged7_70%_Uppersensor_y.dat'}
absabsdiff_modes_SSI2_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\modes_SSI2_Diff_order18\\absabsdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\modes_SSI2_Diff_order18\\absabsdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\modes_SSI2_Diff_order18\\absabsdiff_damaged7_70%_Uppersensor_y.dat'}
abssqudiff_modes_SSI2_Diff_order18_Uppersensor_y = {  \
1:'DamageIndex\\modes_SSI2_Diff_order18\\abssqudiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\modes_SSI2_Diff_order18\\abssqudiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\modes_SSI2_Diff_order18\\abssqudiff_damaged7_70%_Uppersensor_y.dat'}
## modes_SSI4_Diff_order18_Uppersensor_y
absdiff_modes_SSI4_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\modes_SSI4_Diff_order18\\absdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\modes_SSI4_Diff_order18\\absdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\modes_SSI4_Diff_order18\\absdiff_damaged7_70%_Uppersensor_y.dat'}
absabsdiff_modes_SSI4_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\modes_SSI4_Diff_order18\\absabsdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\modes_SSI4_Diff_order18\\absabsdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\modes_SSI4_Diff_order18\\absabsdiff_damaged7_70%_Uppersensor_y.dat'}
abssqudiff_modes_SSI4_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\modes_SSI4_Diff_order18\\abssqudiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\modes_SSI4_Diff_order18\\abssqudiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\modes_SSI4_Diff_order18\\abssqudiff_damaged7_70%_Uppersensor_y.dat'}

#Damage Index Path Definition
## modes_Modal_Diff_Uppersensor_y_less10
absdiff_modes_Modal_Diff_Uppersensor_y_less10 = { \
1:'DamageIndex\\modes_Modal_Diff\\absdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\modes_Modal_Diff\\absdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\modes_Modal_Diff\\absdiff_damaged7_70%_Uppersensor_y_less10.dat'}
absabsdiff_modes_Modal_Diff_Uppersensor_y_less10 = { \
1:'DamageIndex\\modes_Modal_Diff\\absabsdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\modes_Modal_Diff\\absabsdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\modes_Modal_Diff\\absabsdiff_damaged7_70%_Uppersensor_y_less10.dat'}
abssqudiff_modes_Modal_Diff_Uppersensor_y_less10 = {  \
1:'DamageIndex\\modes_Modal_Diff\\abssqudiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\modes_Modal_Diff\\abssqudiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\modes_Modal_Diff\\abssqudiff_damaged7_70%_Uppersensor_y_less10.dat'}
## modes_SSI2_Diff_order18_Uppersensor_y_less10
absdiff_modes_SSI2_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\modes_SSI2_Diff_order18\\absdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\modes_SSI2_Diff_order18\\absdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\modes_SSI2_Diff_order18\\absdiff_damaged7_70%_Uppersensor_y_less10.dat'}
absabsdiff_modes_SSI2_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\modes_SSI2_Diff_order18\\absabsdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\modes_SSI2_Diff_order18\\absabsdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\modes_SSI2_Diff_order18\\absabsdiff_damaged7_70%_Uppersensor_y_less10.dat'}
abssqudiff_modes_SSI2_Diff_order18_Uppersensor_y_less10 = {  \
1:'DamageIndex\\modes_SSI2_Diff_order18\\abssqudiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\modes_SSI2_Diff_order18\\abssqudiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\modes_SSI2_Diff_order18\\abssqudiff_damaged7_70%_Uppersensor_y_less10.dat'}
## modes_SSI4_Diff_order18_Uppersensor_y_less10
absdiff_modes_SSI4_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\modes_SSI4_Diff_order18\\absdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\modes_SSI4_Diff_order18\\absdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\modes_SSI4_Diff_order18\\absdiff_damaged7_70%_Uppersensor_y_less10.dat'}
absabsdiff_modes_SSI4_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\modes_SSI4_Diff_order18\\absabsdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\modes_SSI4_Diff_order18\\absabsdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\modes_SSI4_Diff_order18\\absabsdiff_damaged7_70%_Uppersensor_y_less10.dat'}
abssqudiff_modes_SSI4_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\modes_SSI4_Diff_order18\\abssqudiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\modes_SSI4_Diff_order18\\abssqudiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\modes_SSI4_Diff_order18\\abssqudiff_damaged7_70%_Uppersensor_y_less10.dat'}

In [32]:
# modes_diff_calculation
for i in range(3):
# modes_Modal_Diff
    ## Uppersensor_y
    DSF_undamaged_Path = modesModalSelectedPathdic_Uppersensor_y[0]
    DSF_damaged_Path =   modesModalSelectedPathdic_Uppersensor_y[i+1]  
    modes_prepared_Modal_Uppersensor_y[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_modes_Modal_Diff_Uppersensor_y[i+1]
    absabsdiffPath = absabsdiff_modes_Modal_Diff_Uppersensor_y[i+1]
    abssqudiffPath = abssqudiff_modes_Modal_Diff_Uppersensor_y[i+1]
    modes_prepared_Modal_Uppersensor_y[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
    ## Uppersensor_y_less10
    DSF_undamaged_Path = modesModalSelectedPathdic_Uppersensor_y_less10[0]
    DSF_damaged_Path =   modesModalSelectedPathdic_Uppersensor_y_less10[i+1]  
    modes_prepared_Modal_Uppersensor_y_less10[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_modes_Modal_Diff_Uppersensor_y_less10[i+1]
    absabsdiffPath = absabsdiff_modes_Modal_Diff_Uppersensor_y_less10[i+1]
    abssqudiffPath = abssqudiff_modes_Modal_Diff_Uppersensor_y_less10[i+1]
    modes_prepared_Modal_Uppersensor_y_less10[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
# modes_SSI2_Diff_order18
    ##Uppersensor_y
    DSF_undamaged_Path = modesSSI2_order18Pathdic_Uppersensor_y[0]
    DSF_damaged_Path =   modesSSI2_order18Pathdic_Uppersensor_y[i+1]  
    modes_prepared_SSI2_order18_Uppersensor_y[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_modes_SSI2_Diff_order18_Uppersensor_y[i+1]
    absabsdiffPath = absabsdiff_modes_SSI2_Diff_order18_Uppersensor_y[i+1]
    abssqudiffPath = abssqudiff_modes_SSI2_Diff_order18_Uppersensor_y[i+1]
    modes_prepared_SSI2_order18_Uppersensor_y[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
    ##Uppersensor_y_less10
    DSF_undamaged_Path = modesSSI2_order18Pathdic_Uppersensor_y_less10[0]
    DSF_damaged_Path =   modesSSI2_order18Pathdic_Uppersensor_y_less10[i+1]  
    modes_prepared_SSI2_order18_Uppersensor_y_less10[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_modes_SSI2_Diff_order18_Uppersensor_y_less10[i+1]
    absabsdiffPath = absabsdiff_modes_SSI2_Diff_order18_Uppersensor_y_less10[i+1]
    abssqudiffPath = abssqudiff_modes_SSI2_Diff_order18_Uppersensor_y_less10[i+1]
    modes_prepared_SSI2_order18_Uppersensor_y_less10[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
# modes_SSI4_Diff_order18
    ##Uppersensor_y
    DSF_undamaged_Path = modesSSI4_order18Pathdic_Uppersensor_y[0]
    DSF_damaged_Path =   modesSSI4_order18Pathdic_Uppersensor_y[i+1]  
    modes_prepared_SSI4_order18_Uppersensor_y[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_modes_SSI4_Diff_order18_Uppersensor_y[i+1]
    absabsdiffPath = absabsdiff_modes_SSI4_Diff_order18_Uppersensor_y[i+1]
    abssqudiffPath = abssqudiff_modes_SSI4_Diff_order18_Uppersensor_y[i+1]
    modes_prepared_SSI4_order18_Uppersensor_y[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
    ##Uppersensor_y_less10
    DSF_undamaged_Path = modesSSI4_order18Pathdic_Uppersensor_y_less10[0]
    DSF_damaged_Path =   modesSSI4_order18Pathdic_Uppersensor_y_less10[i+1]  
    modes_prepared_SSI4_order18_Uppersensor_y_less10[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_modes_SSI4_Diff_order18_Uppersensor_y_less10[i+1]
    absabsdiffPath = absabsdiff_modes_SSI4_Diff_order18_Uppersensor_y_less10[i+1]
    abssqudiffPath = abssqudiff_modes_SSI4_Diff_order18_Uppersensor_y_less10[i+1]
    modes_prepared_SSI4_order18_Uppersensor_y_less10[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)

  DamageIndex\modes_Modal_Diff\absdiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\modes_Modal_Diff\absabsdiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\modes_Modal_Diff\abssqudiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\modes_Modal_Diff\absdiff_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\modes_Modal_Diff\absabsdiff_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\modes_Modal_Diff\abssqudiff_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\modes_SSI2_Diff_order18\absdiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\modes_SSI2_Diff_order18\absabsdiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\modes_SSI2_Diff_order18\abssqudiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\modes_SSI2_Diff_order18\absdiff_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\modes_SSI2_Diff_order18\absabsdi

In [17]:
# ModeShapeCurvature_prepared Path definition
## ModeShapeCurvatureModalSelected
ModeShapeCurvatureModalPathdic_Uppersensor_y = { \
0:'ModeShapeCurvature_Modal\\undamaged_Uppersensor_y.dat', \
1:'ModeShapeCurvature_Modal\\damaged7_20%_Uppersensor_y.dat', \
2:'ModeShapeCurvature_Modal\\damaged7_50%_Uppersensor_y.dat', \
3:'ModeShapeCurvature_Modal\\damaged7_70%_Uppersensor_y.dat'}   
ModeShapeCurvatureModalPathdic_Uppersensor_y_less10 = { \
0:'ModeShapeCurvature_Modal\\undamaged_Uppersensor_y_less10.dat', \
1:'ModeShapeCurvature_Modal\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'ModeShapeCurvature_Modal\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'ModeShapeCurvature_Modal\\damaged7_70%_Uppersensor_y_less10.dat'}  
## ModeShapeCurvatureSSI2
ModeShapeCurvatureSSI2Pathdic_Uppersensor_y = \
{0:'ModeShapeCurvature_SSI2_order18\\undamaged_Uppersensor_y.dat', \
1:'ModeShapeCurvature_SSI2_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'ModeShapeCurvature_SSI2_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'ModeShapeCurvature_SSI2_order18\\damaged7_70%_Uppersensor_y.dat'}
ModeShapeCurvatureSSI2Pathdic_Uppersensor_y_less10 = \
{0:'ModeShapeCurvature_SSI2_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'ModeShapeCurvature_SSI2_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'ModeShapeCurvature_SSI2_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'ModeShapeCurvature_SSI2_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
## ModeShapeCurvatureSSI4
ModeShapeCurvatureSSI4Pathdic_Uppersensor_y = \
{0:'ModeShapeCurvature_SSI4_order18\\undamaged_Uppersensor_y.dat', \
1:'ModeShapeCurvature_SSI4_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'ModeShapeCurvature_SSI4_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'ModeShapeCurvature_SSI4_order18\\damaged7_70%_Uppersensor_y.dat'}
ModeShapeCurvatureSSI4Pathdic_Uppersensor_y_less10 = \
{0:'ModeShapeCurvature_SSI4_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'ModeShapeCurvature_SSI4_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'ModeShapeCurvature_SSI4_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'ModeShapeCurvature_SSI4_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
ModeShapeCurvature_Modal_Uppersensor_y = {}
ModeShapeCurvature_Modal_Uppersensor_y_less10 = {}
ModeShapeCurvature_SSI2_Uppersensor_y = {}
ModeShapeCurvature_SSI2_Uppersensor_y_less10 = {}
ModeShapeCurvature_SSI4_Uppersensor_y = {}
ModeShapeCurvature_SSI4_Uppersensor_y_less10 = {}

In [18]:
#Damage Index Path Definition
## ModeShapeCurvature_Modal_Diff_Uppersensor_y
absdiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y = { \
1:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absdiff_damaged7_70%_Uppersensor_y.dat'}
absabsdiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y = { \
1:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absabsdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absabsdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absabsdiff_damaged7_70%_Uppersensor_y.dat'}
abssqudiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y = {  \
1:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\abssqudiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\abssqudiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\abssqudiff_damaged7_70%_Uppersensor_y.dat'}
## ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y
absdiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absdiff_damaged7_70%_Uppersensor_y.dat'}
absabsdiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absabsdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absabsdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absabsdiff_damaged7_70%_Uppersensor_y.dat'}
abssqudiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y = {  \
1:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\abssqudiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\abssqudiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\abssqudiff_damaged7_70%_Uppersensor_y.dat'}
## ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y
absdiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absdiff_damaged7_70%_Uppersensor_y.dat'}
absabsdiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absabsdiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absabsdiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absabsdiff_damaged7_70%_Uppersensor_y.dat'}
abssqudiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\abssqudiff_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\abssqudiff_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\abssqudiff_damaged7_70%_Uppersensor_y.dat'}

#Damage Index Path Definition
## ModeShapeCurvature_Modal_Diff_Uppersensor_y_less10
absdiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y_less10 = { \
1:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absdiff_damaged7_70%_Uppersensor_y_less10.dat'}
absabsdiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y_less10 = { \
1:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absabsdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absabsdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\absabsdiff_damaged7_70%_Uppersensor_y_less10.dat'}
abssqudiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y_less10 = {  \
1:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\abssqudiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\abssqudiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModeShapeCurvature_Modal_Diff\\abssqudiff_damaged7_70%_Uppersensor_y_less10.dat'}
## ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y_less10
absdiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absdiff_damaged7_70%_Uppersensor_y_less10.dat'}
absabsdiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absabsdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absabsdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\absabsdiff_damaged7_70%_Uppersensor_y_less10.dat'}
abssqudiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y_less10 = {  \
1:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\abssqudiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\abssqudiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI2_Diff_order18\\abssqudiff_damaged7_70%_Uppersensor_y_less10.dat'}
## ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y_less10
absdiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absdiff_damaged7_70%_Uppersensor_y_less10.dat'}
absabsdiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absabsdiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absabsdiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\absabsdiff_damaged7_70%_Uppersensor_y_less10.dat'}
abssqudiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\abssqudiff_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\abssqudiff_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModeShapeCurvature_SSI4_Diff_order18\\abssqudiff_damaged7_70%_Uppersensor_y_less10.dat'}

In [19]:
# ModeShapeCurvature_diff_calculation
for i in range(3):
# ModeShapeCurvature_Modal_Diff
    ## Uppersensor_y
    DSF_undamaged_Path = ModeShapeCurvatureModalPathdic_Uppersensor_y[0]
    DSF_damaged_Path =   ModeShapeCurvatureModalPathdic_Uppersensor_y[i+1]  
    ModeShapeCurvature_Modal_Uppersensor_y[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y[i+1]
    absabsdiffPath = absabsdiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y[i+1]
    abssqudiffPath = abssqudiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y[i+1]
    ModeShapeCurvature_Modal_Uppersensor_y[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
    ## Uppersensor_y_less10
    DSF_undamaged_Path = ModeShapeCurvatureModalPathdic_Uppersensor_y_less10[0]
    DSF_damaged_Path =   ModeShapeCurvatureModalPathdic_Uppersensor_y_less10[i+1]  
    ModeShapeCurvature_Modal_Uppersensor_y_less10[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y_less10[i+1]
    absabsdiffPath = absabsdiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y_less10[i+1]
    abssqudiffPath = abssqudiff_ModeShapeCurvature_Modal_Diff_Uppersensor_y_less10[i+1]
    ModeShapeCurvature_Modal_Uppersensor_y_less10[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
# ModeShapeCurvature_SSI2_Diff_order18
    ##Uppersensor_y
    DSF_undamaged_Path = ModeShapeCurvatureSSI2Pathdic_Uppersensor_y[0]
    DSF_damaged_Path =   ModeShapeCurvatureSSI2Pathdic_Uppersensor_y[i+1]  
    ModeShapeCurvature_SSI2_Uppersensor_y[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y[i+1]
    absabsdiffPath = absabsdiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y[i+1]
    abssqudiffPath = abssqudiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y[i+1]
    ModeShapeCurvature_SSI2_Uppersensor_y[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
    ##Uppersensor_y_less10
    DSF_undamaged_Path = ModeShapeCurvatureSSI2Pathdic_Uppersensor_y_less10[0]
    DSF_damaged_Path =   ModeShapeCurvatureSSI2Pathdic_Uppersensor_y_less10[i+1]  
    ModeShapeCurvature_SSI2_Uppersensor_y_less10[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y_less10[i+1]
    absabsdiffPath = absabsdiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y_less10[i+1]
    abssqudiffPath = abssqudiff_ModeShapeCurvature_SSI2_Diff_order18_Uppersensor_y_less10[i+1]
    ModeShapeCurvature_SSI2_Uppersensor_y_less10[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
# ModeShapeCurvature_SSI4_Diff_order18
    ##Uppersensor_y
    DSF_undamaged_Path = ModeShapeCurvatureSSI4Pathdic_Uppersensor_y[0]
    DSF_damaged_Path =   ModeShapeCurvatureSSI4Pathdic_Uppersensor_y[i+1]  
    ModeShapeCurvature_SSI4_Uppersensor_y[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y[i+1]
    absabsdiffPath = absabsdiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y[i+1]
    abssqudiffPath = abssqudiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y[i+1]
    ModeShapeCurvature_SSI4_Uppersensor_y[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)
    ##Uppersensor_y_less10
    DSF_undamaged_Path = ModeShapeCurvatureSSI4Pathdic_Uppersensor_y_less10[0]
    DSF_damaged_Path =   ModeShapeCurvatureSSI4Pathdic_Uppersensor_y_less10[i+1]  
    ModeShapeCurvature_SSI4_Uppersensor_y_less10[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffPath = absdiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y_less10[i+1]
    absabsdiffPath = absabsdiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y_less10[i+1]
    abssqudiffPath = abssqudiff_ModeShapeCurvature_SSI4_Diff_order18_Uppersensor_y_less10[i+1]
    ModeShapeCurvature_SSI4_Uppersensor_y_less10[i].DiffIndexCalculation(absdiffPath,absabsdiffPath,abssqudiffPath)

  DamageIndex\ModeShapeCurvature_Modal_Diff\absdiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModeShapeCurvature_Modal_Diff\absabsdiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModeShapeCurvature_Modal_Diff\abssqudiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModeShapeCurvature_Modal_Diff\absdiff_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\ModeShapeCurvature_Modal_Diff\absabsdiff_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\ModeShapeCurvature_Modal_Diff\abssqudiff_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\ModeShapeCurvature_SSI2_Diff_order18\absdiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModeShapeCurvature_SSI2_Diff_order18\absabsdiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModeShapeCurvature_SSI2_Diff_order18\abssqudiff_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModeShapeCurvatur

In [20]:
# FractionalEnergy_prepared Path definition
## FractionalEnergyModalSelected
FractionalEnergyModalPathdic_Uppersensor_y = { \
0:'FractionalEnergy_Modal\\undamaged_Uppersensor_y.dat', \
1:'FractionalEnergy_Modal\\damaged7_20%_Uppersensor_y.dat', \
2:'FractionalEnergy_Modal\\damaged7_50%_Uppersensor_y.dat', \
3:'FractionalEnergy_Modal\\damaged7_70%_Uppersensor_y.dat'}   
FractionalEnergyModalPathdic_Uppersensor_y_less10 = { \
0:'FractionalEnergy_Modal\\undamaged_Uppersensor_y_less10.dat', \
1:'FractionalEnergy_Modal\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'FractionalEnergy_Modal\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'FractionalEnergy_Modal\\damaged7_70%_Uppersensor_y_less10.dat'}  
## FractionalEnergySSI2
FractionalEnergySSI2Pathdic_Uppersensor_y = \
{0:'FractionalEnergy_SSI2_order18\\undamaged_Uppersensor_y.dat', \
1:'FractionalEnergy_SSI2_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'FractionalEnergy_SSI2_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'FractionalEnergy_SSI2_order18\\damaged7_70%_Uppersensor_y.dat'}
FractionalEnergySSI2Pathdic_Uppersensor_y_less10 = \
{0:'FractionalEnergy_SSI2_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'FractionalEnergy_SSI2_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'FractionalEnergy_SSI2_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'FractionalEnergy_SSI2_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
## FractionalEnergySSI4
FractionalEnergySSI4Pathdic_Uppersensor_y = \
{0:'FractionalEnergy_SSI4_order18\\undamaged_Uppersensor_y.dat', \
1:'FractionalEnergy_SSI4_order18\\damaged7_20%_Uppersensor_y.dat', \
2:'FractionalEnergy_SSI4_order18\\damaged7_50%_Uppersensor_y.dat', \
3:'FractionalEnergy_SSI4_order18\\damaged7_70%_Uppersensor_y.dat'}
FractionalEnergySSI4Pathdic_Uppersensor_y_less10 = \
{0:'FractionalEnergy_SSI4_order18\\undamaged_Uppersensor_y_less10.dat', \
1:'FractionalEnergy_SSI4_order18\\damaged7_20%_Uppersensor_y_less10.dat', \
2:'FractionalEnergy_SSI4_order18\\damaged7_50%_Uppersensor_y_less10.dat', \
3:'FractionalEnergy_SSI4_order18\\damaged7_70%_Uppersensor_y_less10.dat'}
FractionalEnergy_Modal_Uppersensor_y = {}
FractionalEnergy_Modal_Uppersensor_y_less10 = {}
FractionalEnergy_SSI2_Uppersensor_y = {}
FractionalEnergy_SSI2_Uppersensor_y_less10 = {}
FractionalEnergy_SSI4_Uppersensor_y = {}
FractionalEnergy_SSI4_Uppersensor_y_less10 = {}

In [21]:
#Damage Index Path Definition
## ModalStrainEnergy_Modal_Division_Uppersensor_y
beta_Modal_Diff_Uppersensor_y = { \
1:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta_damaged7_70%_Uppersensor_y.dat'}
beta2_Modal_Diff_Uppersensor_y = { \
1:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta2_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta2_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta2_damaged7_70%_Uppersensor_y.dat'}

## ModalStrainEnergy_SSI2_Division_order18_Uppersensor_y
beta_SSI2_Diff_Uppersensor_y ={  \
1:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta_damaged7_70%_Uppersensor_y.dat'}
beta2_SSI2_Diff_Uppersensor_y ={  \
1:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta2_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta2_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta2_damaged7_70%_Uppersensor_y.dat'}

## ModalStrainEnergy_SSI4_Division_order18_Uppersensor_y
beta_SSI4_Diff_Uppersensor_y ={  \
1:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta_damaged7_70%_Uppersensor_y.dat'}
beta2_SSI4_Diff_Uppersensor_y ={  \
1:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta2_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta2_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta2_damaged7_70%_Uppersensor_y.dat'}

#Damage Index Path Definition
## ModalStrainEnergy_Modal_Division_Uppersensor_y_less10
beta_Modal_Diff_Uppersensor_y_less10 = { \
1:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta_damaged7_70%_Uppersensor_y_less10.dat'}
beta2_Modal_Diff_Uppersensor_y_less10 = { \
1:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta2_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta2_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModalStrainEnergy_Modal_Division\\beta2_damaged7_70%_Uppersensor_y_less10.dat'}

## ModalStrainEnergy_SSI2_Division_order18_Uppersensor_y_less10
beta_SSI2_Diff_Uppersensor_y_less10 ={  \
1:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta_damaged7_70%_Uppersensor_y_less10.dat'}
beta2_SSI2_Diff_Uppersensor_y_less10 ={  \
1:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta2_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta2_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModalStrainEnergy_SSI2_Division_order18\\beta2_damaged7_70%_Uppersensor_y_less10.dat'}

## ModalStrainEnergy_SSI4_Division_order18_Uppersensor_y_less10
beta_SSI4_Diff_Uppersensor_y_less10 ={  \
1:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta_damaged7_70%_Uppersensor_y_less10.dat'}
beta2_SSI4_Diff_Uppersensor_y_less10 ={  \
1:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta2_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta2_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\ModalStrainEnergy_SSI4_Division_order18\\beta2_damaged7_70%_Uppersensor_y_less10.dat'}



In [22]:
# FractionalEnergy_diff_calculation
for i in range(3):
# FractionalEnergy_Modal_Diff
    ## Uppersensor_y
    DSF_undamaged_Path = FractionalEnergyModalPathdic_Uppersensor_y[0]
    DSF_damaged_Path =   FractionalEnergyModalPathdic_Uppersensor_y[i+1]  
    FractionalEnergy_Modal_Uppersensor_y[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    betaPath = beta_Modal_Diff_Uppersensor_y[i+1]
    beta2Path = beta2_Modal_Diff_Uppersensor_y[i+1]
    
    FractionalEnergy_Modal_Uppersensor_y[i].DivisionIndexCalculation(betaPath,beta2Path)
    ## Uppersensor_y_less10
    DSF_undamaged_Path = FractionalEnergyModalPathdic_Uppersensor_y_less10[0]
    DSF_damaged_Path =   FractionalEnergyModalPathdic_Uppersensor_y_less10[i+1]  
    FractionalEnergy_Modal_Uppersensor_y_less10[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    betaPath = beta_Modal_Diff_Uppersensor_y_less10[i+1]
    beta2Path = beta2_Modal_Diff_Uppersensor_y_less10[i+1]
   
    FractionalEnergy_Modal_Uppersensor_y_less10[i].DivisionIndexCalculation(betaPath,beta2Path)
# FractionalEnergy_SSI2_Diff
    ##Uppersensor_y
    DSF_undamaged_Path = FractionalEnergySSI2Pathdic_Uppersensor_y[0]
    DSF_damaged_Path =   FractionalEnergySSI2Pathdic_Uppersensor_y[i+1]  
    FractionalEnergy_SSI2_Uppersensor_y[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    betaPath = beta_SSI2_Diff_Uppersensor_y[i+1]
    beta2Path = beta2_SSI2_Diff_Uppersensor_y[i+1]
   
    FractionalEnergy_SSI2_Uppersensor_y[i].DivisionIndexCalculation(betaPath,beta2Path)
    ##Uppersensor_y_less10
    DSF_undamaged_Path = FractionalEnergySSI2Pathdic_Uppersensor_y_less10[0]
    DSF_damaged_Path =   FractionalEnergySSI2Pathdic_Uppersensor_y_less10[i+1]  
    FractionalEnergy_SSI2_Uppersensor_y_less10[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    betaPath = beta_SSI2_Diff_Uppersensor_y_less10[i+1]
    beta2Path = beta2_SSI2_Diff_Uppersensor_y_less10[i+1]
   
    FractionalEnergy_SSI2_Uppersensor_y_less10[i].DivisionIndexCalculation(betaPath,beta2Path)
# FractionalEnergy_SSI4_Diff
    ##Uppersensor_y
    DSF_undamaged_Path = FractionalEnergySSI4Pathdic_Uppersensor_y[0]
    DSF_damaged_Path =   FractionalEnergySSI4Pathdic_Uppersensor_y[i+1]  
    FractionalEnergy_SSI4_Uppersensor_y[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    betaPath = beta_SSI4_Diff_Uppersensor_y[i+1]
    beta2Path = beta2_SSI4_Diff_Uppersensor_y[i+1]
   
    FractionalEnergy_SSI4_Uppersensor_y[i].DivisionIndexCalculation(betaPath,beta2Path)
    ##Uppersensor_y_less10
    DSF_undamaged_Path = FractionalEnergySSI4Pathdic_Uppersensor_y_less10[0]
    DSF_damaged_Path =   FractionalEnergySSI4Pathdic_Uppersensor_y_less10[i+1]  
    FractionalEnergy_SSI4_Uppersensor_y_less10[i] = DamageSensitiveFeature(DSF_undamaged_Path,DSF_damaged_Path)
    betaPath = beta_SSI4_Diff_Uppersensor_y_less10[i+1]
    beta2Path = beta2_SSI4_Diff_Uppersensor_y_less10[i+1]

    FractionalEnergy_SSI4_Uppersensor_y_less10[i].DivisionIndexCalculation(betaPath,beta2Path)

  DamageIndex\ModalStrainEnergy_Modal_Division\beta_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModalStrainEnergy_Modal_Division\beta2_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModalStrainEnergy_Modal_Division\beta_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\ModalStrainEnergy_Modal_Division\beta2_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\ModalStrainEnergy_SSI2_Division_order18\beta_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModalStrainEnergy_SSI2_Division_order18\beta2_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModalStrainEnergy_SSI2_Division_order18\beta_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\ModalStrainEnergy_SSI2_Division_order18\beta2_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\ModalStrainEnergy_SSI4_Division_order18\beta_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\ModalStrai

In [23]:
# Calculate DamgedIndex of Flexibility Matrix
def getCurvature(v,deltax): # Get the Curvature of a vector
    s = v.size
    vc = np.zeros_like(v)
    for j in range(1,s-1):
        vc[j]=(v[j+1]+v[j-1]-2*v[j])/np.square(deltax)
        # At the ends of the structure
    vc[0]=(v[2] + v[0]-2*v[1])/np.square(deltax)
    vc[s-1] = (v[s-1] + v[s-3]-2*v[s-2])/np.square(deltax)
    return vc       
    
class FlexibilityMatrix_Method2:
    def __init__(self,g_undamgedPath,g_damagedPath):
        self.g_undamaged = np.load(g_undamgedPath)
        self.g_damaged = np.load(g_damagedPath)
        self.order,self.sensornumber,self.sensornumber=self.g_damaged.shape
        self.dx = 25/200
        if self.sensornumber == 20:
            self.deltax = 10*self.dx # this will change if we change the sensor number
        elif self.sensornumber == 10:
            self.deltax = 10*self.dx*2
        elif self.sensornumber == 6:
            self.deltax = 10*self.dx*3
    def getDamageIndex(self,absdiffvcPath,absabsdiffvcPath,abssqudiffvcPath):
        self.v_undamaged=np.zeros((self.order,self.sensornumber))
        self.v_damaged=np.zeros((self.order,self.sensornumber))
        self.vc_undamaged = np.zeros((self.order,self.sensornumber))
        self.vc_damaged = np.zeros((self.order,self.sensornumber))
        self.absdiffvc = np.zeros((self.order,self.sensornumber))
        self.absabsdiffvc = np.zeros((self.order,self.sensornumber))
        self.abssqudiffvc = np.zeros((self.order,self.sensornumber))
        for i in range(self.order):
            self.v_undamaged[i] = np.sum(self.g_undamaged[i],axis = 1)
            self.v_damaged[i] = np.sum(self.g_damaged[i],axis=1)
            self.v_undamaged[i]=np.sum(self.g_undamaged[i],axis=1)
            self.v_damaged[i]=np.sum(self.g_damaged[i],axis=1)
            # get the v curvature
            self.vc_undamaged[i]=getCurvature(self.v_undamaged[i],self.deltax)
            self.vc_damaged[i] = getCurvature(self.v_damaged[i],self.deltax)
            self.absdiffvc[i] =abs(self.vc_undamaged[i]-self.vc_damaged[i])
            self.absabsdiffvc[i] = abs(abs(self.vc_undamaged[i])-abs(self.vc_damaged[i]))
            self.abssqudiffvc[i] = abs(np.square(self.vc_undamaged[i])-np.square(self.vc_damaged[i]))
        f = open(absabsdiffvcPath,'w')
        np.savetxt(f,self.absabsdiffvc,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(absabsdiffvcPath))
        f.close()
        f = open(abssqudiffvcPath,'w')
        np.savetxt(f,self.abssqudiffvc,fmt = '%s',delimiter = ',')
        pipe = sys.stdout.write
        pipe('  {} written Finished.\n'.format(abssqudiffvcPath))
        f.close()
    

In [24]:
# Flexibility matrix Path definition
gModalPath_Uppersensor_y = { \
0:'FlexibilityMatrix_Modal\\g_undamaged_Uppersensor_y.npy', \
1:'FlexibilityMatrix_Modal\\g_damaged7_20%_Uppersensor_y.npy', \
2:'FlexibilityMatrix_Modal\\g_damaged7_50%_Uppersensor_y.npy', \
3:'FlexibilityMatrix_Modal\\g_damaged7_70%_Uppersensor_y.npy'}

# GModalPath_Uppersensor_y = { \
# 0:'FlexibilityMatrix_Modal\\G_undamaged_Uppersensor_y.dat', \
# 1:'FlexibilityMatrix_Modal\\G_damaged7_20%_Uppersensor_y.dat', \
# 2:'FlexibilityMatrix_Modal\\G_damaged7_50%_Uppersensor_y.dat', \
# 3:'FlexibilityMatrix_Modal\\G_damaged7_70%_Uppersensor_y.dat'}

gSSI2Path_Uppersensor_y = { \
0:'FlexibilityMatrix_SSI2_order18\\g_undamaged_Uppersensor_y.npy', \
1:'FlexibilityMatrix_SSI2_order18\\g_damaged7_20%_Uppersensor_y.npy', \
2:'FlexibilityMatrix_SSI2_order18\\g_damaged7_50%_Uppersensor_y.npy', \
3:'FlexibilityMatrix_SSI2_order18\\g_damaged7_70%_Uppersensor_y.npy'}

# GSSI2Path_Uppersensor_y = { \
# 0:'FlexibilityMatrix_SSI2_order18\\G_undamaged_Uppersensor_y.dat', \
# 1:'FlexibilityMatrix_SSI2_order18\\G_damaged7_20%_Uppersensor_y.dat', \
# 2:'FlexibilityMatrix_SSI2_order18\\G_damaged7_50%_Uppersensor_y.dat', \
# 3:'FlexibilityMatrix_SSI2_order18\\G_damaged7_70%_Uppersensor_y.dat'}

gSSI4Path_Uppersensor_y = { \
0:'FlexibilityMatrix_SSI4_order18\\g_undamaged_Uppersensor_y.npy', \
1:'FlexibilityMatrix_SSI4_order18\\g_damaged7_20%_Uppersensor_y.npy', \
2:'FlexibilityMatrix_SSI4_order18\\g_damaged7_50%_Uppersensor_y.npy', \
3:'FlexibilityMatrix_SSI4_order18\\g_damaged7_70%_Uppersensor_y.npy'}

# GSSI4Path_Uppersensor_y = { \
# 0:'FlexibilityMatrix_SSI4_order18\\G_undamaged_Uppersensor_y.dat', \
# 1:'FlexibilityMatrix_SSI4_order18\\G_damaged7_20%_Uppersensor_y.dat', \
# 2:'FlexibilityMatrix_SSI4_order18\\G_damaged7_50%_Uppersensor_y.dat', \
# 3:'FlexibilityMatrix_SSI4_order18\\G_damaged7_70%_Uppersensor_y.dat'}

gModalPath_Uppersensor_y_less10 = { \
0:'FlexibilityMatrix_Modal\\g_undamaged_Uppersensor_y_less10.npy', \
1:'FlexibilityMatrix_Modal\\g_damaged7_20%_Uppersensor_y_less10.npy', \
2:'FlexibilityMatrix_Modal\\g_damaged7_50%_Uppersensor_y_less10.npy', \
3:'FlexibilityMatrix_Modal\\g_damaged7_70%_Uppersensor_y_less10.npy'}
# GModalPath_Uppersensor_y_less10 = { \
# 0:'FlexibilityMatrix_Modal\\G_undamaged_Uppersensor_y_less10.dat', \
# 1:'FlexibilityMatrix_Modal\\G_damaged7_20%_Uppersensor_y_less10.dat', \
# 2:'FlexibilityMatrix_Modal\\G_damaged7_50%_Uppersensor_y_less10.dat', \
# 3:'FlexibilityMatrix_Modal\\G_damaged7_70%_Uppersensor_y_less10.dat'}

gSSI2Path_Uppersensor_y_less10 = { \
0:'FlexibilityMatrix_SSI2_order18\\g_undamaged_Uppersensor_y_less10.npy', \
1:'FlexibilityMatrix_SSI2_order18\\g_damaged7_20%_Uppersensor_y_less10.npy', \
2:'FlexibilityMatrix_SSI2_order18\\g_damaged7_50%_Uppersensor_y_less10.npy', \
3:'FlexibilityMatrix_SSI2_order18\\g_damaged7_70%_Uppersensor_y_less10.npy'}

# GSSI2Path_Uppersensor_y_less10 = { \
# 0:'FlexibilityMatrix_SSI2_order18\\G_undamaged_Uppersensor_y_less10.dat', \
# 1:'FlexibilityMatrix_SSI2_order18\\G_damaged7_20%_Uppersensor_y_less10.dat', \
# 2:'FlexibilityMatrix_SSI2_order18\\G_damaged7_50%_Uppersensor_y_less10.dat', \
# 3:'FlexibilityMatrix_SSI2_order18\\G_damaged7_70%_Uppersensor_y_less10.dat'}


gSSI4Path_Uppersensor_y_less10 = { \
0:'FlexibilityMatrix_SSI4_order18\\g_undamaged_Uppersensor_y_less10.npy', \
1:'FlexibilityMatrix_SSI4_order18\\g_damaged7_20%_Uppersensor_y_less10.npy', \
2:'FlexibilityMatrix_SSI4_order18\\g_damaged7_50%_Uppersensor_y_less10.npy', \
3:'FlexibilityMatrix_SSI4_order18\\g_damaged7_70%_Uppersensor_y_less10.npy'}

# GSSI4Path_Uppersensor_y_less10 = { \
# 0:'FlexibilityMatrix_SSI4_order18\\G_undamaged_Uppersensor_y_less10.dat', \
# 1:'FlexibilityMatrix_SSI4_order18\\G_damaged7_20%_Uppersensor_y_less10.dat', \
# 2:'FlexibilityMatrix_SSI4_order18\\G_damaged7_50%_Uppersensor_y_less10.dat', \
# 3:'FlexibilityMatrix_SSI4_order18\\G_damaged7_70%_Uppersensor_y_less10.dat'}

In [28]:
#Damage Index Path Definition
## FlexibilityMatrix_Modal_Diff_Uppersensor_y
absabsdiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y = { \
1:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absabsdiffvc_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absabsdiffvc_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absabsdiffvc_damaged7_70%_Uppersensor_y.dat'}
absdiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y = { \
1:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absdiffvc_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absdiffvc_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absdiffvc_damaged7_70%_Uppersensor_y.dat'}
abssqudiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y = {  \
1:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\abssqudiffvc_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\abssqudiffvc_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\abssqudiffvc_damaged7_70%_Uppersensor_y.dat'}
## FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y
absabsdiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absabsdiffvc_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absabsdiffvc_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absabsdiffvc_damaged7_70%_Uppersensor_y.dat'}
absdiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absdiffvc_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absdiffvc_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absdiffvc_damaged7_70%_Uppersensor_y.dat'}
abssqudiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y = {  \
1:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\abssqudiffvc_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\abssqudiffvc_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\abssqudiffvc_damaged7_70%_Uppersensor_y.dat'}
## FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y
absabsdiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absabsdiffvc_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absabsdiffvc_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absabsdiffvc_damaged7_70%_Uppersensor_y.dat'}
absdiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absdiffvc_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absdiffvc_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absdiffvc_damaged7_70%_Uppersensor_y.dat'}
abssqudiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\abssqudiffvc_damaged7_20%_Uppersensor_y.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\abssqudiffvc_damaged7_50%_Uppersensor_y.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\abssqudiffvc_damaged7_70%_Uppersensor_y.dat'}

#Damage Index Path Definition
## FlexibilityMatrix_Modal_Diff_Uppersensor_y_less10
absabsdiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y_less10 = { \
1:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absabsdiffvc_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absabsdiffvc_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absabsdiffvc_damaged7_70%_Uppersensor_y_less10.dat'}
absdiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y_less10 = { \
1:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absdiffvc_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absdiffvc_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\absdiffvc_damaged7_70%_Uppersensor_y_less10.dat'}
abssqudiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y_less10 = {  \
1:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\abssqudiffvc_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\abssqudiffvc_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\FlexibilityMatrix_Modal_Diff\\abssqudiffvc_damaged7_70%_Uppersensor_y_less10.dat'}
## FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y_less10
absabsdiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absabsdiffvc_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absabsdiffvc_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absabsdiffvc_damaged7_70%_Uppersensor_y_less10.dat'}
absdiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absdiffvc_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absdiffvc_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\absdiffvc_damaged7_70%_Uppersensor_y_less10.dat'}
abssqudiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y_less10 = {  \
1:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\abssqudiffvc_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\abssqudiffvc_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI2_Diff_order18\\abssqudiffvc_damaged7_70%_Uppersensor_y_less10.dat'}
## FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y_less10
absabsdiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absabsdiffvc_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absabsdiffvc_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absabsdiffvc_damaged7_70%_Uppersensor_y_less10.dat'}
absdiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absdiffvc_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absdiffvc_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\absdiffvc_damaged7_70%_Uppersensor_y_less10.dat'}
abssqudiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y_less10 ={  \
1:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\abssqudiffvc_damaged7_20%_Uppersensor_y_less10.dat', \
2:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\abssqudiffvc_damaged7_50%_Uppersensor_y_less10.dat', \
3:'DamageIndex\\FlexibilityMatrix_SSI4_Diff_order18\\abssqudiffvc_damaged7_70%_Uppersensor_y_less10.dat'}

In [29]:
FlexibilityMatrix_Modal_Uppersensor_y={}
FlexibilityMatrix_Modal_Uppersensor_y_less10 = {}
FlexibilityMatrix_SSI2_Uppersensor_y = {}
FlexibilityMatrix_SSI2_Uppersensor_y_less10 ={}
FlexibilityMatrix_SSI4_Uppersensor_y = {}
FlexibilityMatrix_SSI4_Uppersensor_y_less10 ={}
# FlexibilityMatrix_diff_calculation
for i in range(3):
# FlexibilityMatrix_Modal_Diff
    ## Uppersensor_y
    DSF_undamaged_Path = gModalPath_Uppersensor_y[0]
    DSF_damaged_Path =   gModalPath_Uppersensor_y[i+1]  
    FlexibilityMatrix_Modal_Uppersensor_y[i] = FlexibilityMatrix_Method2(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffvcPath = absdiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y[i+1]
    absabsdiffvcPath = absabsdiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y[i+1]
    abssqudiffvcPath = abssqudiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y[i+1]
    FlexibilityMatrix_Modal_Uppersensor_y[i].getDamageIndex(absdiffvcPath,absabsdiffvcPath,abssqudiffvcPath)
    ## Uppersensor_y_less10
    DSF_undamaged_Path = gModalPath_Uppersensor_y_less10[0]
    DSF_damaged_Path =   gModalPath_Uppersensor_y_less10[i+1]  
    FlexibilityMatrix_Modal_Uppersensor_y_less10[i] = FlexibilityMatrix_Method2(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffvcPath = absdiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y_less10[i+1]
    absabsdiffvcPath = absabsdiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y_less10[i+1]
    abssqudiffvcPath = abssqudiffvc_FlexibilityMatrix_Modal_Diff_Uppersensor_y_less10[i+1]
    FlexibilityMatrix_Modal_Uppersensor_y_less10[i].getDamageIndex(absdiffvcPath,absabsdiffvcPath,abssqudiffvcPath)
# FlexibilityMatrix_SSI2_Diff_order18
    ##Uppersensor_y
    DSF_undamaged_Path = gSSI2Path_Uppersensor_y[0]
    DSF_damaged_Path =   gSSI2Path_Uppersensor_y[i+1]  
    FlexibilityMatrix_SSI2_Uppersensor_y[i] = FlexibilityMatrix_Method2(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffvcPath = absdiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y[i+1]
    absabsdiffvcPath = absabsdiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y[i+1]
    abssqudiffvcPath = abssqudiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y[i+1]
    FlexibilityMatrix_SSI2_Uppersensor_y[i].getDamageIndex(absdiffvcPath,absabsdiffvcPath,abssqudiffvcPath)
    ##Uppersensor_y_less10
    DSF_undamaged_Path = gSSI2Path_Uppersensor_y_less10[0]
    DSF_damaged_Path =   gSSI2Path_Uppersensor_y_less10[i+1]  
    FlexibilityMatrix_SSI2_Uppersensor_y_less10[i] = FlexibilityMatrix_Method2(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffvcPath = absdiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y_less10[i+1]
    absabsdiffvcPath = absabsdiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y_less10[i+1]
    abssqudiffvcPath = abssqudiffvc_FlexibilityMatrix_SSI2_Diff_order18_Uppersensor_y_less10[i+1]
    FlexibilityMatrix_SSI2_Uppersensor_y_less10[i].getDamageIndex(absdiffvcPath,absabsdiffvcPath,abssqudiffvcPath)
# FlexibilityMatrix_SSI4_Diff_order18
    ##Uppersensor_y
    DSF_undamaged_Path = gSSI4Path_Uppersensor_y[0]
    DSF_damaged_Path =   gSSI4Path_Uppersensor_y[i+1]  
    FlexibilityMatrix_SSI4_Uppersensor_y[i] = FlexibilityMatrix_Method2(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffvcPath = absdiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y[i+1]
    absabsdiffvcPath = absabsdiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y[i+1]
    abssqudiffvcPath = abssqudiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y[i+1]
    FlexibilityMatrix_SSI4_Uppersensor_y[i].getDamageIndex(absdiffvcPath,absabsdiffvcPath,abssqudiffvcPath)
    ##Uppersensor_y_less10
    DSF_undamaged_Path = gSSI4Path_Uppersensor_y_less10[0]
    DSF_damaged_Path =   gSSI4Path_Uppersensor_y_less10[i+1]  
    FlexibilityMatrix_SSI4_Uppersensor_y_less10[i] = FlexibilityMatrix_Method2(DSF_undamaged_Path,DSF_damaged_Path)
    absdiffvcPath = absdiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y_less10[i+1]
    absabsdiffvcPath = absabsdiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y_less10[i+1]
    abssqudiffvcPath = abssqudiffvc_FlexibilityMatrix_SSI4_Diff_order18_Uppersensor_y_less10[i+1]
    FlexibilityMatrix_SSI4_Uppersensor_y_less10[i].getDamageIndex(absdiffvcPath,absabsdiffvcPath,abssqudiffvcPath)

  DamageIndex\FlexibilityMatrix_Modal_Diff\absabsdiffvc_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\FlexibilityMatrix_Modal_Diff\abssqudiffvc_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\FlexibilityMatrix_Modal_Diff\absabsdiffvc_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\FlexibilityMatrix_Modal_Diff\abssqudiffvc_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\FlexibilityMatrix_SSI2_Diff_order18\absabsdiffvc_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\FlexibilityMatrix_SSI2_Diff_order18\abssqudiffvc_damaged7_20%_Uppersensor_y.dat written Finished.
  DamageIndex\FlexibilityMatrix_SSI2_Diff_order18\absabsdiffvc_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\FlexibilityMatrix_SSI2_Diff_order18\abssqudiffvc_damaged7_20%_Uppersensor_y_less10.dat written Finished.
  DamageIndex\FlexibilityMatrix_SSI4_Diff_order18\absabsdiffvc_damaged7_20%_Uppersensor_y.dat written Fi

In [30]:
FlexibilityMatrix_Modal_Uppersensor_y[1].absdiffvc[0]

array([2.74188661e-05, 2.74188661e-05, 3.49805880e-05, 4.37134706e-05,
       2.40216726e-04, 3.43128956e-05, 7.58636740e-05, 8.94295455e-05,
       1.03490778e-04, 1.22584353e-04, 7.66845745e-05, 3.03761592e-06,
       6.99266984e-05, 1.19576501e-04, 1.48224695e-04, 1.53788173e-04,
       1.36249384e-04, 9.77254807e-05, 4.21490931e-05, 4.21490931e-05])